In [1]:
import numpy as np
import os

localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '60G', 'spark.executor.memory': '60G'}, 
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4609352/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4609352/


23/09/01 09:20:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0767:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230901-0919-0.2.113-cf32652c5077.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from oligogenic_model import prep_pl_for_skat, run_skat_log, run_skat_for_mts

In [ ]:
mts = []

for i in list(range(1,26)):
    
    mt = hl.read_matrix_table(
        localfs_path+'after_pca_no_filters_gts_vs_pol_subseted'+str(i)+'.mt'
    )
    
    mt = mt.checkpoint(
        '/net/ascratch/people/plggosborcz/mts/after_pca_no_filters_gts_vs_pol_subseted'+str(i)+'.mt'
    )
    
    mts.append(mt)

In [ ]:
len(mts)

### Run SKAT for pl vs controls

#### annotate each of the mts with nearest genes and chekpoint:

In [11]:
mts = []

for i in list(range(1,26)):
    
    mt = hl.read_matrix_table(
        localfs_path+'after_pca_no_filters_gts_vs_pol_subseted'+str(i)+'.mt'
    )
    
    mts.append(mt)

In [12]:
len(mts)

25

In [ ]:
mts = prep_pl_for_skat(mts)

## Run SKAT for polish samples (9 different subsets) and GTS on scores, for all genes:

In [ ]:
mts = []

for i in list(range(18)):
    mt = hl.read_matrix_table(localfs_path+'_pl_'+str(i)+'ann.mt')
    mts.append(mt)

In [ ]:
len(mts)

In [ ]:
for i in list(range(18)):
    mts[i].write(
        '/net/ascratch/people/plggosborcz/mts/pl_'+str(i)+'ann_3k.mt'
    )

In [13]:
mts = []

for i in list(range(25)):
    mts.append(
        hl.read_matrix_table(
            '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/pl_'+str(i)+'ann_3k.mt'
        )
    )

FatalError: HailException: No file or directory found at /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/pl_0ann_3k.mt

Java stack trace:
is.hail.utils.HailException: No file or directory found at /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/pl_0ann_3k.mt
	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:17)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:17)
	at is.hail.utils.package$.fatal(package.scala:78)
	at is.hail.expr.ir.RelationalSpec$.readMetadata(AbstractMatrixTableSpec.scala:33)
	at is.hail.expr.ir.RelationalSpec$.readReferences(AbstractMatrixTableSpec.scala:69)
	at is.hail.variant.ReferenceGenome$.fromHailDataset(ReferenceGenome.scala:560)
	at is.hail.backend.spark.SparkBackend.pyLoadReferencesFromDataset(SparkBackend.scala:618)
	at jdk.internal.reflect.GeneratedMethodAccessor11.invoke(Unknown Source)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(Delegat

In [ ]:
def run_skat_for_mts(mt_list):
    
    skat_tables = []
    plots = []
    options = []

    for idx, mtx in enumerate(mt_list):

        for pc in list(range(21)):
            
            print('running with '+str(pc)+' PCs and the pruned scores option')
            
            if os.path.exists(localfs_path+'skat_table_'+str(pc)+'_scores_'+str(idx)+'.mt'):
            
                skat_table = hl.read_table(
                        localfs_path+'skat_table_'+str(pc)+'_scores_'+str(idx)+'.mt'
                    )
                
                skat_tables.append(skat_table)
                
                plots.append(
                    hl.plot.qq(
                        skat_table.p_value
                    )
                )
                
                options.append(
                        'PCs '+str(pc)+' pruned scores subset '+str(idx)
                    )
                
            else:
                
                try:
                    skat_table, qq_plot = run_skat_log(
                            mtx,
                            pc,
                            'scores_no_filter',
                            '_part_'+str(idx)
                    )

                    skat_tables.append(skat_table)
                    plots.append(qq_plot)

                    options.append(
                            'PCs '+str(pc)+' pruned scores'+'eur subset '+str(idx)
                        )

                except Exception as e:
                    print('failed PCs '+str(pc)+' pruned scores'+'eur subset '+str(idx))
                    print(type(e))
                

            print('running with '+str(pc)+' PCs and the just subsetet scores')
            
            if os.path.exists(localfs_path+'skat_table_'+str(pc)+'_scores_no_filter_parts_'+str(idx)+'.mt'):
            
                skat_table = hl.read_table(
                    localfs_path+'skat_table_'+str(pc)+'_scores_no_filter_parts_'+str(idx)+'.mt'
                )
                
                skat_tables.append(skat_table)
                
                plots.append(hl.plot.qq(
                    skat_table.p_value
                ))
                options.append(
                        'PCs '+str(pc)+' subseted scores subset '+str(idx)
                    )
                
            else:
                try:
                    skat_table_no_filt, qq_plot_no_filt = run_skat_log(
                        mtx,
                        pc,
                        'scores_no_filter',
                        '_parts_'+str(idx)
                    )

                    skat_tables.append(skat_table_no_filt)
                    plots.append(qq_plot_no_filt)
                    options.append(
                        'PCs '+str(pc)+' no filters subseted scores subset '+str(idx)
                    )

                except Exception as e:
                    print('failed PCs '+str(pc)+' no filters subseted scores'+'eur subset '+str(idx))
                    print(type(e))
            
    return(skat_tables, plots, options)

In [17]:
skat_tables, plots, options = run_skat_for_mts(mts)

running with 0 PCs and the pruned scores option


2023-08-27 21:54:04.081 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-08-27 21:54:07.940 Hail: INFO: wrote table with 17405 rows in 489 partitions to /localfs/4609352/tmp_hail/persist_tableG7PsB05jFx
    Total size: 520.56 KiB
    * Rows: 520.55 KiB
    * Globals: 11.00 B
    * Smallest partition: 25 rows (733.00 B)
    * Largest partition:  58 rows (1.73 KiB)
2023-08-27 21:54:10.228 Hail: INFO: wrote table with 17405 rows in 489 partitions to /localfs/4609352/skat_table_0_scores_no_filter_part_0.mt
2023-08-27 21:54:11.275 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CSF2RA""",12,7.79e+02,-6.05e-09,0
"""DHRSX""",45,2.25e+03,0.00e+00,0
"""OR7D2""",11,1.69e+03,0.00e+00,0
"""PLS3""",1,6.30e-22,0.00e+00,0
"""REPS2""",1,2.63e-22,0.00e+00,0
"""RSRP1""",1,1.51e-21,0.00e+00,0
"""ZBED1""",12,5.39e+02,0.00e+00,0
"""ZNF121""",1,1.01e-21,0.00e+00,0


2023-08-27 21:54:13.624 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-27 21:54:17.920 Hail: INFO: Ordering unsorted dataset with network shuffle


running with 0 PCs and the just subsetet scores


...

2023-08-27 21:59:01.683 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-27 21:59:05.176 Hail: INFO: Ordering unsorted dataset with network shuffle


running with 1 PCs and the pruned scores option
failed PCs 1 pruned scoreseur subset 3
<class 'LookupError'>
running with 1 PCs and the just subsetet scores
failed PCs 1 no filters subseted scoreseur subset 3
<class 'LookupError'>
running with 2 PCs and the pruned scores option
failed PCs 2 pruned scoreseur subset 3
<class 'LookupError'>
running with 2 PCs and the just subsetet scores
failed PCs 2 no filters subseted scoreseur subset 3
<class 'LookupError'>
running with 3 PCs and the pruned scores option
failed PCs 3 pruned scoreseur subset 3
<class 'LookupError'>
running with 3 PCs and the just subsetet scores
failed PCs 3 no filters subseted scoreseur subset 3
<class 'LookupError'>
running with 4 PCs and the pruned scores option
failed PCs 4 pruned scoreseur subset 3
<class 'LookupError'>
running with 4 PCs and the just subsetet scores
failed PCs 4 no filters subseted scoreseur subset 3
<class 'LookupError'>
running with 5 PCs and the pruned scores option
failed PCs 5 pruned scoreseu

2023-08-27 21:59:56.191 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-08-27 21:59:59.695 Hail: INFO: wrote table with 17366 rows in 489 partitions to /localfs/4609352/tmp_hail/persist_tableaX0oUpz3dA
    Total size: 519.16 KiB
    * Rows: 519.15 KiB
    * Globals: 11.00 B
    * Smallest partition: 25 rows (740.00 B)
    * Largest partition:  53 rows (1.58 KiB)
ERROR:root:KeyboardInterrupt while sending command.            (214 + 12) / 489]
Traceback (most recent call last):
  File "/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/net/software/testing/software/Python/3.10.4-GCCcore-11.3.0/lib/p

KeyboardInterrupt: 

2023-08-27 22:00:01.958 Hail: INFO: wrote table with 17366 rows in 489 partitions to /localfs/4609352/skat_table_0_scores_no_filter_part_4.mt


In [20]:
skat_tables = []
lambdas = []

for x in list(range(26)):
    for i in list(range(10)):
        
        if os.path.exists(
                localfs_path+'skat_table_'+str(x)+'_scores_no_filter_parts_'+str(i)+'.mt'
            ):
            skat_table = hl.read_table(
                    localfs_path+'skat_table_'+str(x)+'_scores_no_filter_parts_'+str(i)+'.mt'
                )
            print('skat_table_'+str(x)+'_scores_'+str(i)+'.mt')

            print(hl.lambda_gc(skat_table.p_value))
            lambdas.append(hl.lambda_gc(skat_table.p_value))
            skat_tables.append(skat_table)

            p = hl.plot.qq(skat_table.p_value)
            show(p)

skat_table_0_scores_0.mt
1.4066690564224429


2023-08-28 09:01:08.643 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-28 09:01:12.300 Hail: INFO: Ordering unsorted dataset with network shuffle


skat_table_0_scores_1.mt
1.4847376593403245


2023-08-28 09:01:15.691 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-28 09:01:19.360 Hail: INFO: Ordering unsorted dataset with network shuffle


skat_table_0_scores_2.mt
1.2904615897325735


2023-08-28 09:01:22.851 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-28 09:01:26.483 Hail: INFO: Ordering unsorted dataset with network shuffle


skat_table_0_scores_3.mt
1.402008668160759


2023-08-28 09:01:29.959 Hail: INFO: Ordering unsorted dataset with network shuffle
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-28 09:01:33.558 Hail: INFO: Ordering unsorted dataset with network shuffle


In [12]:
skat_tables = []

for x in list(range(20)):
    
    lambdas = []
    
    for i in list(range(10)):
        
        if os.path.exists(
                localfs_path+'skat_table_'+str(x)+'_scores_no_filter_parts_'+str(i)+'.mt'
            ):
            skat_table = hl.read_table(
                    localfs_path+'skat_table_'+str(x)+'_scores_no_filter_parts_'+str(i)+'.mt'
                )
            print('skat_table_'+str(x)+'_scores_no_filter_'+str(i)+'.mt')

            lambdas.append(hl.lambda_gc(skat_table.p_value))
            skat_tables.append(skat_table)
            
    print(lambdas)
    print(np.median(np.array(lambdas)))         

skat_table_0_scores_no_filter_0.mt
skat_table_0_scores_no_filter_1.mt
skat_table_0_scores_no_filter_2.mt
skat_table_0_scores_no_filter_3.mt
skat_table_0_scores_no_filter_4.mt
skat_table_0_scores_no_filter_5.mt
skat_table_0_scores_no_filter_6.mt
skat_table_0_scores_no_filter_7.mt


skat_table_0_scores_no_filter_8.mt


skat_table_0_scores_no_filter_9.mt
[1.0363764059264284, 1.0804163707004464, 1.1256049610631171, 1.0671375932319822, 1.021461297350317, 1.0485846732234518, 0.9976877610316498, 1.084040956828418, 1.0895133991119659, 1.071932855019685]
1.0695352241258336
skat_table_1_scores_no_filter_0.mt
skat_table_1_scores_no_filter_1.mt
skat_table_1_scores_no_filter_2.mt
skat_table_1_scores_no_filter_3.mt
skat_table_1_scores_no_filter_4.mt
skat_table_1_scores_no_filter_5.mt
skat_table_1_scores_no_filter_6.mt
skat_table_1_scores_no_filter_7.mt
skat_table_1_scores_no_filter_8.mt
skat_table_1_scores_no_filter_9.mt
[0.6354069727678273, 0.9673587764472519, 0.7112788948533134, 0.6010283422455839, 0.5034486389198399, 0.6964737112332731, 0.6190903597185196, 0.5972682690169613, 0.6586745165364165, 0.4910163456138358]
0.6272486662431734
skat_table_2_scores_no_filter_0.mt
skat_table_2_scores_no_filter_1.mt
skat_table_2_scores_no_filter_2.mt
skat_table_2_scores_no_filter_3.mt
skat_table_2_scores_no_filter_4.mt
ska

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/net/software/testing/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

### Now run the same tests but in selected genes for best parameters

In [6]:
#mts = []

#for i in list(range(10)):
#    mts.append(
#        hl.read_matrix_table(
#            '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/pl_'+str(i)+'annot.mt'
#        )
# )
    
    
mts = []

for i in list(range(10)):
    mt = hl.read_matrix_table(localfs_path+'_pl_'+str(i)+'ann.mt')
    mts.append(mt)
    
    
skat_tables = []
plots = []

gene_list = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']

In [ ]:
for idx, mt in enumerate(mts):
    mt = mt.filter_rows(hl.literal(gene_list).contains(mt.nearest_genes_20kb))
    #skat_table, qq_plot = run_skat_log(mt, 5, 'scores_no_filter', '_polish_without_'+str(idx))
    #seed1
    skat_table, qq_plot = run_skat_log(mt, 6, 'scores_no_filter', '_gts_genes_'+str(idx))
    skat_tables.append(skat_table)
    plots.append(qq_plot)

2023-08-24 08:57:54.765 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-08-24 08:57:56.252 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/4540692/tmp_hail/persist_tableFcqknxf6W2
    Total size: 5.65 KiB
    * Rows: 5.64 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (55.00 B)
    * Largest partition:  1 rows (85.00 B)
2023-08-24 08:57:57.815 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/4540692/skat_table_6_scores_no_filter_gts_genes_0.mt
2023-08-24 08:57:58.325 Hail: INFO: Coerced sorted dataset
2023-08-24 08:57:58.331 Hail: INFO: Coerced dataset with out-of-order partitions.


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHADL""",100,4.45e+03,7.07e-05,0
"""MAOA""",5,9.90e+01,4.48e-04,0
"""HTR2A""",246,4.01e+03,9.87e-04,0
"""DRD2""",222,3.71e+03,2.16e-03,0
"""SLITRK5""",132,1.42e+03,4.19e-03,0
"""CDH9""",448,4.11e+03,1.26e-02,0
"""PCDH1""",113,3.43e+03,1.28e-02,0
"""SORCS3""",1324,1.73e+04,1.65e-02,0


2023-08-24 08:58:00.257 Hail: INFO: Coerced sorted dataset
2023-08-24 08:58:00.259 Hail: INFO: Coerced dataset with out-of-order partitions.
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-24 08:58:02.863 Hail: INFO: Coerced sorted dataset
2023-08-24 08:58:02.865 Hail: INFO: Coerced dataset with out-of-order partitions.


2023-08-24 09:06:38.190 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-08-24 09:06:39.118 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/4540692/tmp_hail/persist_tableZ4HORALRTA
    Total size: 5.65 KiB
    * Rows: 5.64 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (55.00 B)
    * Largest partition:  1 rows (85.00 B)
2023-08-24 09:06:39.751 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/4540692/skat_table_6_scores_no_filter_gts_genes_1.mt
2023-08-24 09:06:40.068 Hail: INFO: Coerced sorted dataset
2023-08-24 09:06:40.070 Hail: INFO: Coerced dataset with out-of-order partitions.


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""SORCS3""",1334,2.19e+04,5.09e-04,0
"""CHADL""",96,3.63e+03,5.77e-04,0
"""PCDH10""",235,4.04e+03,2.53e-03,0
"""CDH9""",452,4.62e+03,5.96e-03,0
"""MAOA""",5,7.93e+01,7.64e-03,0
"""KCNQ5""",1081,1.28e+04,9.24e-03,0
"""PCDH1""",112,3.56e+03,1.06e-02,0
"""HTR2A""",246,3.13e+03,1.35e-02,0


2023-08-24 09:06:41.140 Hail: INFO: Coerced sorted dataset
2023-08-24 09:06:41.142 Hail: INFO: Coerced dataset with out-of-order partitions.
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-08-24 09:06:42.865 Hail: INFO: Coerced sorted dataset
2023-08-24 09:06:42.867 Hail: INFO: Coerced dataset with out-of-order partitions.


In [13]:
skat_results = skat_tables[0]

skat_results = skat_results.select(
    skat_results.size
)

field_names = []

for i in list(range(9)):
    field_names.append('p_value_'+str(i))

skat_results = skat_results.annotate(
    **{
        f'p_value_{idx}': skat_tables[idx][skat_results.id].p_value
        for idx in list(range(10))
    }
)

skat_results.export('results-with-synthetic-controls-2.tsv')

2023-08-24 10:27:00.332 Hail: INFO: merging 76 files totalling 8.3K...
2023-08-24 10:27:00.350 Hail: INFO: while writing:
    results-with-synthetic-controls-2.tsv
  merge time: 17.992ms
